In [5]:
import pandas as pd
import os,sys
import numpy as np

In [7]:
sch_file = 'data/vapor.library.2'
cdm = {} # dictionary holding the CDM tables and Schr. table (that will become the sensor_configuration table)
dtypes={ 'station_id': np.int32, 'latitude':str, 'longitude':str, 'altitude':str,
                'rstype':'S4','datetime':np.int,'date_flag':'S2','Station Name':'S60'}

names=list(dtypes.keys())

# Metadata Schroeder -> columns: Index(['station_id', 'rstype', 'datetime', 'date_flag', 'Station Name', 'latitude', 'longitude', 'altitude'], dtype='object')
sch_meta = pd.read_csv(sch_file,  sep=':', header=0, dtype=dtypes, names=names )        
for l in 'latitude','longitude','altitude':
    sch_meta[l]=pd.to_numeric(sch_meta.pop(l), errors='coerce')

sch_instr=pd.read_csv('data/vapor.instruments.all' , sep=':', names=('sensor_id','comments') )



/tmp/ipykernel_2100399/1251255101.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  'rstype':'S4','datetime':np.int,'date_flag':'S2','Station Name':'S60'}


In [10]:
sch_meta['rstype'] = [s.decode('utf-8') for s in sch_meta.rstype ]

In [11]:
sch_meta.head()

,station_id,rstype,datetime,date_flag,Station Name,latitude,longitude,altitude
0,1001,V11,194800000000,b'+-',"b'Jan Mayen - ""Atlantic City"", Norway'",71.0120,-8.4620,-888.0
1,1001,V11,194900000000,b'+-',b'Jan Mayen - Old Meteorological Station (Gaml...,71.0142,-8.4602,39.0
2,1001,V13,196200000000,b'>-',"b'ENJA Jan Mayen - Jan Mayensfield, Norway'",70.9396,-8.6681,9.0
3,1001,V15,0,b'+/',"b'ENJA Jan Mayen - Jan Mayensfield, Norway'",70.9396,-8.6681,8.0
4,1001,V15,197510000000,b'#o',"b'ENJA Jan Mayen - Jan Mayensfield, Norway'",70.9396,-8.6681,8.0


In [35]:
sch_meta_red = sch_meta[sch_meta['datetime'] > 0 ]
all_sensors = np.unique(sch_meta_red['rstype'])

In [22]:
all_sensors

array([' WGN', ' W_P', ' XXX', ..., 'Zqr', '_P', 'u06'], dtype=object)

In [36]:
sch_meta_red.head()

,station_id,rstype,datetime,date_flag,Station Name,latitude,longitude,altitude
0,1001,V11,194800000000,b'+-',"b'Jan Mayen - ""Atlantic City"", Norway'",71.0120,-8.4620,-888.0
1,1001,V11,194900000000,b'+-',b'Jan Mayen - Old Meteorological Station (Gaml...,71.0142,-8.4602,39.0
2,1001,V13,196200000000,b'>-',"b'ENJA Jan Mayen - Jan Mayensfield, Norway'",70.9396,-8.6681,9.0
4,1001,V15,197510000000,b'#o',"b'ENJA Jan Mayen - Jan Mayensfield, Norway'",70.9396,-8.6681,8.0
5,1001,V18,197611241200,b'+<',"b'ENJA Jan Mayen - Jan Mayensfield, Norway'",70.9396,-8.6681,8.0


In [62]:
min_dates, station_ids, station_names, lats, lons = [],[],[], [],[]

for s in all_sensors:
    df_sensor = sch_meta_red[ sch_meta_red.rstype == s ]  ### all stations using this sensor 
    intro_date = min( df_sensor.datetime ) ### earliest date of adoption
    df_stations = df_sensor[df_sensor.datetime ==  intro_date ]
    
    min_dates.append( str(min( df_stations[ df_stations.rstype == s ].datetime ))[:4]  )
    station_ids.append(df_stations.station_id.values)
    station_names.append( [s.decode('utf-8') for s in df_stations['Station Name'].values] ) 
    
    lats.append(df_stations.latitude.values) 
    lons.append(df_stations.longitude.values) 
    

1080

In [66]:
df_intr = pd.DataFrame.from_dict( {'sensor':all_sensors , 
                                   'mindate': min_dates,
                                   'station_id' : station_ids ,
                                   'station_name' : station_names ,
                                   'latitude':lats,
                                   'longitude':lons
                                  } )

df_intr = df_intr.loc[df_intr.mindate < '2020']

In [67]:
df_intr = df_intr.sort_values(by=['mindate'])

In [68]:
df_intr

,sensor,mindate,station_id,station_name,latitude,longitude
925,XXX,1893,[72434],"[St. Louis, MO]",[999.9999],[999.9999]
485,NF1,1893,[72434],"[St. Louis, MO]",[38.63],[-90.2]
526,NUt,1894,[74492],"[Boston (Milton - Blue Hill Observatory), MA]",[42.21],[-71.11]
486,NF3,1896,[74492],"[Boston (Milton - Blue Hill Observatory), MA]",[42.21],[-71.11]
509,NU1,1897,[74492],"[Boston (Milton - Blue Hill Observatory), MA]",[42.21],[-71.11]
...,...,...,...,...,...,...
314,KUuW,2012,[73357],"[(made up station ID) Norman, OK - 449 MHz DeT...",[35.236],[-97.463]
884,VNW,2012,[72712],"[KCAR [IB] Caribou AP - NWS Office, 810 N. Mai...",[46.8683],[-68.0136]
97,DGN,2012,[7758],"[(made up station ID) INRA, San Giuliano, Cors...",[42.2852],[9.5234]
668,SGG,2012,[6610],"[Payerne, Switzerland]",[46.8131],[6.9437]


In [70]:
#for i in df_intr.station_id.values:
#    if len(i) >1:
#        print(i)

[72326 72334 72344 72405 72421 72435 72439 72451 72456 72524 72547 72554
 72562 72660 72734 72745]
[3104 3348 3767 3963]
[71261 71262 72877]
[71261 72877]
[4339 7145]
[72222 72794 74509 74707]
[26066 27612]
[72226 72327 72353 72553 72564 72677 72753 72785 74463 74498 74567 74570
 74747]
[26066 27612]
[72327 72353 72493 72553 72734 72753]
[72327 72753]
[7180 7240 7257 7280 7354]
[20049 26781 28440 28901 29634]
[3005 3743 3811 3953]
[3005 3743]
[94120 94294 94326 94335 94510 94568 94677 94772 94865 94995 94996]
[20049 21982 23330 23933 24688 25594 26781 27037 27553 28698 30230 33837
 35121 36177 37054]
[3005 3743]
[94120 94326 94335 94510 94568 94677 94772 94995]
[47401 47412 47582 47590 47600 47646 47744 47778 47827 47971]
[ 3005  3743  3808  3917  3953  8495 88890 93115]
[20049 20107 20292 20353 20667 20674 20891 21432 21824 21965 21982 22113
 22271 22522 22550 22802 23022 23074 23146 23205 23274 23330 23418 23472
 23552 23804 23884 23921 23933 23955 24266 24343 24507 24641 24688 24790

In [ ]:
def get_data_year(df, year):
    
    d = df.loc[df['datetime']==year]
    